# Analysis of 3 datasets from Chicago

Database Querying using SQLite

Author: Javier Serrano Jodral
Date: 10/10/2025

## Initial set up

Install required libraries and adjust printing with prettytable

In [1]:
#%pip install pandas 
#%pip install ipython-sql prettytable   # install magic module and prettytable

import prettytable
prettytable.DEFAULT = 'DEFAULT'

Load pandas and sqlite 3 libraries. Establish connection to / create FinalDB.db

In [2]:
import pandas, sqlite3

con = sqlite3.connect('FinalDB.db')



Load SQL magic module in order to execute SQL queries

In [3]:
%load_ext sql

Read the 3 CSV datasets from website as DF. Then, load their data to DB as tables.

In [4]:
census_df = pandas.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/FinalModule_Coursera_V5/data/ChicagoCensusData.csv?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkDB0201ENSkillsNetwork20127838-2021-01-01")
schools_df = pandas.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/FinalModule_Coursera_V5/data/ChicagoPublicSchools.csv?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkDB0201ENSkillsNetwork20127838-2021-01-01")
crime_df = pandas.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/FinalModule_Coursera_V5/data/ChicagoCrimeData.csv?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkDB0201ENSkillsNetwork20127838-2021-01-01")

try:
    census_df.to_sql("CENSUS_DATA", con)
    schools_df.to_sql("CHICAGO_PUBLIC_SCHOOLS", con)
    crime_df.to_sql("CHICAGO_CRIME_DATA", con)
except:
    print("Tables already exist in the database.")

Tables already exist in the database.


Connect SQL magic module with FinalDB.db database

In [5]:
%sql sqlite:///FinalDB.db

## Problems suggested

### Problem 1. Find the total number of crimes recorded in the CRIME table.

In [6]:
# Just count the number of rows of the CHICAGO_CRIME_DATA table, because each dataset entry describes one crime of diverse nature.

%sql SELECT COUNT(*) FROM CHICAGO_CRIME_DATA;

# There are 533 rows, which means that there are 533 crimes recorded.

 * sqlite:///FinalDB.db
Done.


COUNT(*)
533


### Problem 2. List community area names and numbers with per capita income less than 11000.

In [7]:
%%sql
-- Simply use a WHERE clause to filter rows with PER_CAPITA_INCOME < 11000 from the CENSUS_DATA table.
-- There are 4 community areas that meet the condition.

SELECT COMMUNITY_AREA_NAME, COMMUNITY_AREA_NUMBER
FROM CENSUS_DATA
WHERE PER_CAPITA_INCOME < 11000;

 * sqlite:///FinalDB.db
Done.


COMMUNITY_AREA_NAME,COMMUNITY_AREA_NUMBER
West Garfield Park,26.0
South Lawndale,30.0
Fuller Park,37.0
Riverdale,54.0


### Problem 3. List all case numbers for crimes involving minors (children are not considered minors for the purposes of crime analysis)

In [8]:
%%sql
-- We proceed by searching for the keyword "MINOR" in the DESCRIPTION column of the CHICAGO_CRIME_DATA table.
-- With this technique, 2 crimes involving minors are retrieved.

SELECT * 
FROM CHICAGO_CRIME_DATA 
WHERE DESCRIPTION LIKE "%MINOR%";

 * sqlite:///FinalDB.db
Done.


index,ID,CASE_NUMBER,DATE,BLOCK,IUCR,PRIMARY_TYPE,DESCRIPTION,LOCATION_DESCRIPTION,ARREST,DOMESTIC,BEAT,DISTRICT,WARD,COMMUNITY_AREA_NUMBER,FBICODE,X_COORDINATE,Y_COORDINATE,YEAR,LATITUDE,LONGITUDE,LOCATION
514,3987219,HL266884,2005-03-31,024XX N CLARK ST,2210,LIQUOR LAW VIOLATION,SELL/GIVE/DEL LIQUOR TO MINOR,CONVENIENCE STORE,1,0,2333,19,43.0,7.0,22,1172680.0,1916483.0,2005,41.92626872,-87.64089934,"(41.926268719, -87.640899336)"
515,3266814,HK238408,2004-03-13,093XX S STONY ISLAND AVE,2230,LIQUOR LAW VIOLATION,ILLEGAL CONSUMPTION BY MINOR,ALLEY,1,0,413,4,8.0,48.0,22,1188539.0,1843379.0,2004,41.72530099,-87.58496589,"(41.72530099, -87.584965887)"


### Problem 4. List all kidnapping crimes involving a child

In [9]:
%%sql
-- Like before, I use a DESCRIPTION filter to find child, and I combine it with a filter on PRIMARY_TYPE to select only kidnapping crimes.
-- Just 1 crime involving a child is retrieved.

SELECT * 
FROM CHICAGO_CRIME_DATA 
WHERE PRIMARY_TYPE = "KIDNAPPING" AND DESCRIPTION LIKE "%CHILD%";

 * sqlite:///FinalDB.db
Done.


index,ID,CASE_NUMBER,DATE,BLOCK,IUCR,PRIMARY_TYPE,DESCRIPTION,LOCATION_DESCRIPTION,ARREST,DOMESTIC,BEAT,DISTRICT,WARD,COMMUNITY_AREA_NUMBER,FBICODE,X_COORDINATE,Y_COORDINATE,YEAR,LATITUDE,LONGITUDE,LOCATION
520,5276766,HN144152,2007-01-26,050XX W VAN BUREN ST,1792,KIDNAPPING,CHILD ABDUCTION/STRANGER,STREET,0,0,1533,15,29.0,25.0,20,1143050.0,1897546.0,2007,41.87490841,-87.75024931,"(41.874908413, -87.750249307)"


### Problem 5. List the kind of crimes that were recorded at schools. (No repetitions)

In [10]:
%%sql
-- Return all the unique values of PRIMARY_TYPE from the CHICAGO_CRIME_DATA table, where the LOCATION_DESCRIPTION contains the word "SCHOOL".
-- We get 6 different types of crimes.

SELECT DISTINCT PRIMARY_TYPE
FROM CHICAGO_CRIME_DATA 
WHERE LOCATION_DESCRIPTION LIKE "%SCHOOL%";


 * sqlite:///FinalDB.db
Done.


PRIMARY_TYPE
BATTERY
CRIMINAL DAMAGE
NARCOTICS
ASSAULT
CRIMINAL TRESPASS
PUBLIC PEACE VIOLATION


### Problem 6. List the type of schools along with the average safety score for each type.

In [11]:
%%sql
-- Using GROUP BY statement on the "Elementary, Middle, or High School" column of the CHICAGO_PUBLIC_SCHOOLS table, and then applying the AVG() function to the SAFETY_SCORE column, we get the average safety score for each type of school.
-- There are 3 types of schools. Their respective average safety scores are shown in descending order.

SELECT "Elementary, Middle, or High School", AVG(SAFETY_SCORE) AS AVERAGE_SAFETY_SCORE
FROM CHICAGO_PUBLIC_SCHOOLS
GROUP BY "Elementary, Middle, or High School"
ORDER BY AVERAGE_SAFETY_SCORE DESC;


 * sqlite:///FinalDB.db
Done.


"Elementary, Middle, or High School",AVERAGE_SAFETY_SCORE
HS,49.62352941176471
ES,49.52038369304557
MS,48.0


### Problem 7. List 5 community areas with highest % of households below poverty line

In [12]:
%%sql
-- The key here is to utilize the ORDER BY clause to sort the rows by PERCENT_HOUSEHOLDS_BELOW_POVERTY in descending order, and then limit the output to 5 rows using the LIMIT clause.
-- We notice that the 5 community areas with the highest percentage of households below the poverty line have a percentage between 56.5% and 42.4%

SELECT *
FROM CENSUS_DATA
ORDER BY PERCENT_HOUSEHOLDS_BELOW_POVERTY DESC
LIMIT 5;

 * sqlite:///FinalDB.db
Done.


index,COMMUNITY_AREA_NUMBER,COMMUNITY_AREA_NAME,PERCENT_OF_HOUSING_CROWDED,PERCENT_HOUSEHOLDS_BELOW_POVERTY,PERCENT_AGED_16__UNEMPLOYED,PERCENT_AGED_25__WITHOUT_HIGH_SCHOOL_DIPLOMA,PERCENT_AGED_UNDER_18_OR_OVER_64,PER_CAPITA_INCOME,HARDSHIP_INDEX
53,54.0,Riverdale,5.8,56.5,34.6,27.5,51.5,8201,98.0
36,37.0,Fuller Park,3.2,51.2,33.9,26.6,44.9,10432,97.0
67,68.0,Englewood,3.8,46.6,28.0,28.5,42.5,11888,94.0
28,29.0,North Lawndale,7.4,43.1,21.2,27.6,42.7,12034,87.0
26,27.0,East Garfield Park,8.2,42.4,19.6,21.3,43.2,12961,83.0


### Problem 8. Which community area is most crime prone? Display the community area number only.

In [13]:
%%sql
-- The community area most proone to crime is that with the highest number of crimes recorded, which translates to the highest number of entries in the CHICAGO_CRIME_DATA table. We opt for a GROUP BY statement on the COMMUNITY_AREA_NUMBER column, and then count the number of occurrences of each community area. Finally, we sort the results in descending order and limit the output to 1 row.
-- The most crime prone community area is number 25.

SELECT COMMUNITY_AREA_NUMBER
FROM CHICAGO_CRIME_DATA 
GROUP BY COMMUNITY_AREA_NUMBER
ORDER BY COUNT(COMMUNITY_AREA_NUMBER) DESC
LIMIT 1;

 * sqlite:///FinalDB.db
Done.


COMMUNITY_AREA_NUMBER
25.0


### Problem 9. Use a sub-query to find the name of the community area with highest hardship index

In [ ]:
%%sql
-- We solve this problem by doing a subquery that retrieves the maximum value of HARDSHIP_INDEX from the CENSUS_DATA table, and then using this value to filter the rows of the same table.
-- Notice that the community returned is the same as the first one from the Problem 7, which shows a correlation between PERCENT_HOUSEHOLDS_BELOW_POVERTY and HARDSHIP_INDEX.

SELECT COMMUNITY_AREA_NUMBER
FROM CENSUS_DATA
WHERE HARDSHIP_INDEX = (SELECT MAX(HARDSHIP_INDEX) 
                        FROM CENSUS_DATA);

 * sqlite:///FinalDB.db
Done.


COMMUNITY_AREA_NUMBER
54.0


### Problem 10. Use a sub-query to determine the Community Area Name with most number of crimes?

In [15]:
%%sql
-- Basically, we are asked about the same question as the Problem 8, but proceeding with a sub-query and printing the community area name instead of the number. We can even reuse the query from Problem 8 as the sub-query here. The community area name is only available in the CENSUS_DATA table. We use the community area number as foreign key to connect the two tables.
-- We realise that community area number 25 is Austin.

SELECT COMMUNITY_AREA_NAME, COMMUNITY_AREA_NUMBER
FROM CENSUS_DATA
WHERE COMMUNITY_AREA_NUMBER = (SELECT COMMUNITY_AREA_NUMBER
                                FROM CHICAGO_CRIME_DATA
                                GROUP BY COMMUNITY_AREA_NUMBER
                                ORDER BY COUNT(COMMUNITY_AREA_NUMBER) DESC
                                LIMIT 1);


 * sqlite:///FinalDB.db
Done.


COMMUNITY_AREA_NAME,COMMUNITY_AREA_NUMBER
Austin,25.0
